In [23]:
from androguard.misc import AnalyzeAPK
import matplotlib.pyplot as plt
import numpy as np
from random import randrange
from IPython.display import clear_output
import json
import glob
import time
import sys
import cv2
import os

In [24]:
#!pip3 install opencv-python

In [25]:
def OneD2TwoD_resize(image_pixels):
    #print(len(image_pixels))
    image_pixels = np.array(image_pixels)
    
    w = int(image_pixels.shape[0]**0.5)
    h = int(image_pixels.shape[0]/w)
    
    twoD_image = image_pixels[:w*h].reshape(h,w)
    
    twoD_image = twoD_image.astype('float32')
    
    #image_resized = resize(twoD_image, (64, 64),anti_aliasing=True)
    if(twoD_image.shape[0]>64):
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_AREA)
    else:
        image_resized = cv2.resize(twoD_image, (64,64), interpolation = cv2.INTER_CUBIC)
    return image_resized

In [26]:
def dex2Image(dx,d):
    #Convert APIs to pixel values
    blue_channel_stuff, API_calls_pixels = API_calls2pixels(dx)
    
    red_cahnnel = OneD2TwoD_resize(API_calls_pixels)  #[API_calls_pixels + opcodes_pixls][0]

    #Convert Opcodes to pixel values
    opcodes_pixls = list(opcodes2pixels(dx))
    blue_channel_stuff += opcodes_pixls
    
    #Convert strings to pixel values
    blue_channel_stuff += strings2pixels(d)
    
    return red_cahnnel, blue_channel_stuff

def API_calls2pixels(dx):
    imprtnt_API_calls = []
    nrml_API_calls = []
    for api in dx.get_external_classes():
        api_tmp = api.get_vm_class()
        for i in api_tmp.get_methods():
            api_call = str(i).split('(')[0] # ignoring parameters and return type.
            if(api_call in API_calls_dict):
                imprtnt_API_calls.append(API_calls_dict[api_call])
            else:
                nrml_API_calls.append(sum(api_call.encode())%256)
    return imprtnt_API_calls, nrml_API_calls

def opcodes2pixels(dx):
    opcodes = []
    for method in dx.get_methods():
        if method.is_external():
            continue
        m = method.get_method()
        for ins in m.get_instructions():
          #if(ins.get_op_value not in opcodes):
          opcodes.append(ins.get_op_value())
    opcodes = list(set(opcodes))
    return opcodes

def strings2pixels(d):
    string_pixels = []
    all_strings = d[0].get_strings()
    [string_pixels.append((sum(strng.encode()))%256) for strng in all_strings]
    return string_pixels

In [27]:
def manifest2Image(a):
    #Permissions
    permissions = a.get_permissions() + a.get_declared_permissions()
    imprtnt_permissions = []
    nrml_permissions = []
    for permission in permissions:
        perm = permission.split('.')[-1]
        # main key word (permission) is oftenly(always) comes at last.
        if(perm in malicious_permissions):
            imprtnt_permissions.append(malicious_permissions[perm])
        elif(perm in decleared_permissions):
            imprtnt_permissions.append(decleared_permissions[perm])
        else:
            nrml_permissions += list(permission.encode())
    
    #Activities
    activities = a.get_activities()
    imprtnt_activities = []
    nrml_activities = []
    for activity in activities:
        act = activity.split('.')[-1]
        if(act in malicous_activities):
            imprtnt_activities.append(malicous_activities[act])
        else:
            nrml_activities += list(activity.encode())
            
    #Services
    services = a.get_services()
    imprtnt_services = []
    nrml_services = []
    for service in services:
        srvc = service.split('.')[-1]
        if(srvc in malicious_services):
            imprtnt_services.append(malicious_services[srvc])
        else:
            nrml_services += list(service.encode())
            
    #Recivers
    receivers = a.get_receivers()
    imprtnt_receivers = []
    nrml_receivers = []
    for receiver in receivers:
        recevr = receiver.split('.')[-1]
        if(recevr in malicious_recivers):
            imprtnt_receivers.append(recevr)
        else:
            nrml_receivers += list(receiver.encode())
            
    #Providers
    providers = a.get_providers()
    nrml_providers = []
    for provider in providers:
        nrml_providers += list(provider.encode())
        
    #Intents
    imprtnt_intents = []
    nrml_intents = []
    manifest_list = {'permissions':permissions,'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    intents_itemtype = {'activity' : activities, 'service': services, 'receiver':receivers, 'provider':providers}
    for itemtype, listt in intents_itemtype.items():
        for item in listt:
            try:
                for intnts in a.get_intent_filters(itemtype, item).values():
                    for intnt in intnts:
                        if(intnt in malicious_intents):
                            imprtnt_intents.append(malicious_intents[intnt])
                        else:
                            nrml_intents += list(intnt.encode())
            except:
                pass
    blue_channel_stuff = imprtnt_permissions + imprtnt_activities + imprtnt_services + imprtnt_intents + imprtnt_receivers
    green_channel = nrml_permissions + nrml_activities + nrml_services + nrml_providers + nrml_intents
    green_channel = OneD2TwoD_resize(green_channel)
    return green_channel, blue_channel_stuff

In [28]:
def apk2image(a, d, dx):
    green_channel, blue_channel_stuff1 = manifest2Image(a)
    
    red_channel, blue_channel_stuff2 = dex2Image(dx, d)
    
    blue_channel = blue_channel_stuff1 + blue_channel_stuff2
    blue_channel = OneD2TwoD_resize(blue_channel)
    
    image = cv2.merge((blue_channel, green_channel, red_channel))
    image = image.astype(dtype='uint8')
    return image

In [29]:
malware_APKs_list = os.listdir('/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Malware_APK_Top_10')
print(len(malware_APKs_list))

6086


In [30]:
temp_list = os.listdir('/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Malware_Images_V3_top_10_Families/')

#print(len(temp_list_12K))
print(len(temp_list))

temp_list2 = []
for a in temp_list:
    temp_list2.append(a[:-4])

print(len(temp_list2))

14533
14533


In [31]:
for ap in temp_list2:
    apk = ap+'.apk'
    if(apk in malware_APKs_list):
        malware_APKs_list.remove(apk)
        
print(len(malware_APKs_list))

4349


In [43]:
malware_imges_path = '/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Malware_Images_V3_top_10_Families/'
malware_APKs_path = '/Users/asim/Desktop/Asim_Research/Adversarial_ML/AndroZoo/APK_Files/Malware/Malware_APK_Top_10/'

for sha in malware_APKs_list:
    print('Analysing :', sha)

    try:
        #Androguard
        a,d,dx = AnalyzeAPK(os.path.join(malware_APKs_path + sha))

        img = apk2image(a,d,dx)

        #Saving to local directory
        cv2.imwrite(os.path.join(malware_imges_path + sha[:-4] + '.jpg'), img)
    except:
        pass
    

Analysing : 8D2C32D89133CB96C9E0A6924B28E6D9749B4A318329762999FC4A3A6C25CB69.apk
Analysing : 7A93EF274CAC79AAB0CF7A418C874F3F9C389648A0332C51C179774EBBD22521.apk
Analysing : 8ED6A43F954603F6AD93C166C9204FEFEED6FD82CF8EED65FF65C78845D4171F.apk
Analysing : C138FFD940BCD6DDFC496A37F7B39768FCBE9FBAB8C70E70D5BCD870F7B41F9C.apk
Analysing : 5DEEAE3C78718382FBDD3C3DEAFC6693052D1DC28AFD6AC8B4BCDA35C72A9D3A.apk
Analysing : A011EC3C6CB23C9FD2B07073278823C9A9FF3E91C06F65D62AE1C003DEF598F1.apk
Analysing : F2E08633CC4B7B0591CDF081B8EF07D3EA6DD2E814A8CF98DF347162FE558585.apk
Analysing : 92073FE917BE82B2AD02AA25DE4AF6678D3D01149A66F382C26461FED3BE05FF.apk
Analysing : 95D04FAA6E68A609C6C5403C6489A796A3CA3E9BB72A82DE63A4159D8DA8743C.apk
Analysing : 6D98C5704400BCAB7397B7731812933157A807F3B65270796B19B3CEB6B09CE4.apk
Analysing : 576F70AC27B7762759EE93D66AEAE96310023BD3802960F94F2C0B29674098DB.apk
Analysing : 955F64B291425B11B474ECA16639779019E33ABE48ED9D2F8C4DE2F7812E51CE.apk
Analysing : C41F15D40E4BB827

Name 'android:versionName' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:minSdkVersion' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:targetSdkVersion' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest

Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removin

Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:enabled' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:priority' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:priority' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:name' starts with 'android:' prefix! The Manifest seems to be broken? Removing prefix.
Name 'android:priority' starts with 'android:' prefix! The Manifest seems to be 

Analysing : 3EA4FC2B91E89B58FE48CA18D9C9FF2406A0928338B5E318704CFD3D6ADD46CF.apk
Analysing : F3AA5AB52F62002537DC8624511C9B089415B439D382AA1C74E1B1A6BA42A716.apk


...

Analysing : E694580768A7AFF3DE1AEFE506A746C25CD57A0799064A5F37A71A873079280D.apk
Analysing : 3E59CF390F47188624001655B2340B831B09D02898C08DD9526A1517D9022CBC.apk
Analysing : AD0FA7CBF2CF2B742DA0F152C17E9BC9517A3D33884EDF6EC2B80A89179FEDC5.apk
Analysing : DF447588B45E5926BA5753B9632B783E27D0F07483D8E41779BB879E14919678.apk
Analysing : B701C0553084B731FAB49680074CA1C4FDE0158031A79EF784B3B5458A6EB2B2.apk
Analysing : 8920F112BD5DBD9E1AE159B10331B2D247C9BAE945715F96F7C1F6D56783C196.apk
Analysing : 38501545C0697F0D4A3C476C5C67839B75F9E9FFB6D6C95F7E5613718133EFC2.apk
Analysing : 39499CF93B333A877798BD4012FE26535359E79A0EFA10CA638AE7F394643217.apk
Analysing : 38E09B0AD2A80AF8C1FD4FBAA720F9867BCCBBCF2F8ABC4D0E4E3DB534B470CC.apk
Analysing : B7B8730FD558CB5754280F453A143006F62E8DF3EAA1F6EAC6808E8A568E214E.apk
Analysing : 7336779628717AB0173785E1C69F5A71161F081BF86EB76FC0B6D159258118EE.apk
Analysing : CB66B76E6417AE12847B1A37A73A939489679E02F1EB4C6FDD251677DA6C7401.apk
Analysing : E77E11C9250263B6

Requested API Level could not be found, using 19 instead


Analysing : C52C8DC416723EF844D52D3C0F4B8B19CE62E4658DFC7261E8E6CEC3AEF7CEC9.apk
Analysing : 1A2F2623D83F54ECFBA2583E54F5445563C781355BC0A1EE491C2DBAA7C7220A.apk
Analysing : 8B0F2091ACB8300A412E0C2F6C4DB67A6E5ECD8AAAB2D76174A97D2E0CBF6431.apk
Analysing : 367287974D89BCBC81C4B1A4A97983A372544A9F1BE673DBF6877CA61620A043.apk
Analysing : 6A1B8A2DAA28DF907BC2054613904C049CF420DF682523024EC2F0D84B8F660D.apk
Analysing : C0FE6FC92B3C26B09F272EB97454A50091F06290FA6F66F9C19493D48C06445D.apk
Analysing : F40D7A185FE20F9D40C1944E616BCD223F1AEEBB9C81DBA234F69C1121FE3830.apk
Analysing : EC9C61D35F45804CA362AB13DDFED32BCC44726792E99BA7962788F07166DCE4.apk
Analysing : 158EBA3E12A0969F990EEAADAC80BD441B68F73C1E0210AD46B0D79B9D1D2FF9.apk
Analysing : BD64ED6952BC27B8469A18D26E193579CECB50410761DE9F0B56CE298775E9E2.apk
Analysing : 717F39C675BFE4829EB90609CB4CCCD6312E989563CEE2DF448C22097972E75F.apk
Analysing : 7E1B21209F6B7D2DB16868A93A755FFE779763BCF31791B01F9594DBB0207D25.apk
Analysing : B6541366AA83DE6A

Requested API Level could not be found, using 10 instead


Analysing : 15B2E826410A777BC962BC999FC11564A89902614F8A1EF85983DCCEB40FE042.apk
Analysing : 97BA3DDA50A219248036CD6EEE52C9D0D80D6A7B9ED1A3F600B681D734D21B28.apk
Analysing : F7E59A295962305CB3E36E746F4BB6443E8C2153E117D7E9796CC4A7D9DE81D2.apk
Analysing : 342A5C8057DA111A9DE80C223A6907CE80062F7AB64825E0EF7C90469043F98E.apk
Analysing : EA683B6FE321581E14DC1C4ECEBEEFC9FB481618EB6A80977AFBF14A78DA7D66.apk
Analysing : 2E4CBDDAE7ECC467F74902EE549BD8D300D7DD5BB3991ACD704BC1DF1C011909.apk
Analysing : C9676A92FF58E81CDD540AD0E5E0E7C6293867C42232A8945E5441E5DDCAE629.apk
Analysing : CCAE5C19B58CDCEC964EB5FCAC7A0EB7D8515DE8506706A8A5DE8E767273693A.apk
Analysing : 3ACE7F6EF36A4B6A51F5B719E8E92C66B22381579FC190EB0E0B953BF6930968.apk
Analysing : 2EA759E2F55AB67A7741FC503E98EAD42DB171CC2859E81E6C5B8805CFE78CC6.apk
Analysing : 4FB4B14CEC8E8B152A55C2660ABE39CCE2C9D650DA0EF1583510EEE9F5FFB86E.apk
Analysing : 0FC4A4C9A05A1E111E83564B7BC9AE67F19D1B1B67A4F3F23913BB8121BDDA18.apk
Analysing : 4D9D5C226E4CF0DF

## Multi Processing

In [0]:
malware_imges_path = '../AndroZoo/Malware/AndroGuard+Predefined_Dictionary/Malware_Images_4K_Onwards/'
malware_APKs_path = '../AndroZoo/Malware/Malware_APKs_geq35/'

def apk2Image_main(sha):
    print(sha)
    
    try:
        #Androguard
        a,d,dx = AnalyzeAPK(os.path.join(malware_APKs_path + sha))

        img = apk2image(a,d,dx)

        #Saving to local directory
        cv2.imwrite(os.path.join(malware_imges_path + sha[:-4] + '.jpg'), img)
    except:
        pass

In [21]:
import concurrent.futures
start_time = time.time()
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(apk2Image_main, malware_APKs_list)
    
print(time.time()-start_time)

NameError: name 'apk2Image_main' is not defined

# Important API Calls

In [32]:
# Risky API_calls and methods
API_calls_dict = {
    'Ljavax/sql/ConnectionEvent;-><init>':250,
    'Ljava/nio/channels/WritableByteChannel;->close':240,
    'Landroid/service/carrier/CarrierService;->stopSelf':230,
    'Landroid/opengl/Matrix;->getClass':220,
    'Landroid/view/ViewStructure;->setCheckable':210,
    'Landroid/text/method/BaseKeyListener;->getInputType':200,
    'Landroid/provider/MediaStore$Images$Media;->wait':190,
    'Ljava/lang/Runtime;->exec':180,
    'Ljava/lang/System;->loadLibrary':180,
    'Landroid/widget/AdapterView;->refreshDrawableState':170,
    'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':170,

    'Ljava/io/BufferedOutputStream;-><init>':160,
    'Ljava/io/FileOutputStream;-><init>':160,

    'Landroid/app/PendingIntent;->send':150,
    'Landroid/app/AlarmManager;->Set':150,
    'Landroid/app/NativeActivity;->getVolumeControlStream':140,
    'Landroid/app/AcitivityManager;->killBackgroudProcess':140,
    
    'Landroid/content/pm/PacakageManager;->removePackageFromPrefe':130,
    'Landroid/content/pm/PacakageManager;->getInastallerPackageName':130,
    'Landroid/content/pm/PacakageManager;->getInstalledPackages':130,
    'Landroid/content/pm/PacakageManager;->getInstalledApplications':130,
    'Landroid/content/Intent;->startActivity':120,
    'Landroid/content/Intent;->getAction':120,
    'Landroid/content/Intent;->setDataAndType':120,
    'Landroid/content/ContentResolver;->delete':115,
    'Landroid/content/ContentResolver;->update':115,
    'Landroid/content/ContentResolver;->insert':115,
    'Landroid/content/Context;->getPackageManager':115,
    'Landroid/content/BroadcastReceiver;->abortBroadcast':115,

    'Landroid/database/sqlote/SQLiteDatabase;->execSQL':110,

    'Landroid/media/MediaRecorder;->MediaRecorder':100,
    'Ljava/net/HttpURLConnection;->connect':95,
    'Ljava/net/URLConnection-;>connect':95,
    'Lorg/apache/http/impl/client;->DefaultHttpClient':90,

    'Ljavax/crypto/Cipher;->getInstance':85,
    'Ljavax/crypto/Cipher;->Init':85,
    'Ljavax/crypto/Cipher;->doFinal':85,

    'Landroid/telephony/TelephonyManager;->getDataActivity':80,
    'Landroid/telephony/TelephonyManager;->getCallState':80,
    'Landroid/telephony/TelephonyManager;->getLine1Number':80,
    'Landroid/telephony/TelephonyManager;->getCellLocation':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/TelephonyManager;->getDeviceId':80,
    'Landroid/telephony/TelephonyManager;->getNetworkType':80,
    'Landroid/telephony/TelephonyManager;->getSimOperator':80,
    'Landroid/telephony/TelephonyManager;->getSimSerialNumber':80,
    'Landroid/telephony/TelephonyManager;->getSimState':80,
    'Landroid/telephony/TelephonyManager;->getSubscriberId':80,
    'Landroid/telephony/SmsManager;->sendTextMessage':75,
    'Landroid/telephony/SmsManager;->sendMultipartTextMessage':75,
    'Landroid/telephony/SmsManager;->sendDataMessage':75,
    'Landroid/telephony/SmsMessage;->getServiceCenterAddress':75,
    'Landroid/telephony/gsm/SmsManager;->sendMultipartTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendDataMessage':70,
    'Landroid/telephony/gsm/SmsManager;->sendTextMessage':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayOriginatingAddress':70,
    'Landroid/telephony/gsm/SmsManager;->getDisplayMessageBody':70,
    'Landroid/telephony/PhoneStateListener;->onCallStateChanged':65,

    'Ldalvik/system/DexClassLoader;->loadClass':60,
    'Ldalvik/system/PathClassLoader;->loadClass':60,
    'Landroid/net/wifi/WifiInfo;->getSupplicantState':50,
    'Landroid/net/wifi/WifiManager;->getConnectionInfo':40,
    'Landroid/ContentResolver;->query':30, 
    'Landroid/location/LocationManager;->getLastKnownLocation':20,
}

## Collected following Refrences

Android Malware Identification Through Visual Exploration of Disassembly Files. 2017
International Journal of Network Security, Vol.20, No.6, PP.1061-1073, Nov. 2018 (DOI: 10.6633/IJNS.201811 20(6).07) 1061

In [33]:
# Risky API_calls and methods
malicious_apis_1 = {'Ljava/lang/Runtime->exec': 220, 'Landroid/content/Intent->startActivity': 210,
                 'Landroid/app/PendingIntent->send': 200, 'Landroid/app/AlarmManager->Set': 200,
                 'Landroid/content/pm/PacakageManager->removePackageFromPrefe': 190,
                 'Landroid/database/sqlote/SQLiteDatabase->execSQL': 180,
                 'Landroid/content/ContentResolver->delete': 170,
                 'Landroid/app/AcitivityManager->killBackgroudProcess': 160,
                 'Landroid/media/MediaRecorder->MediaRecorder': 150, 'Ljava/net/HttpURLConnection->connect': 140,
                 'Ljava/net/URLConnection->connect': 140, 'Lorg/apache/http/impl/client->DefaultHttpClient': 140,
                 'Landroid/content/BroadcastReceiver->abortBroadcast': 63,
                 'Landroid/telephony/PhoneStateListener->onCallStateChanged': 130,
                 'Landroid/content/Intent->getAction': 120, 'Ljavax/crypto/Cipher->getInstance': 110,
                 'Ljavax/crypto/Cipher->Init': 110, 'Ljavax/crypto/Cipher->doFinal': 110,
                 'Landroid/telephony/TelephonyManager->getLine1Number': 100,
                 'Landroid/content/pm/PacakageManager->getInastallerPackageName': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledPackages': 90,
                 'Landroid/content/pm/PacakageManager->getInstalledApplications': 90,
                 'Landroid/location/LocationManager->getLastKnownLocation': 80,
                 'Landroid/telephony/TelephonyManager->getCellLocation': 80,
                 'Landroid/telephony/TelephonyManager->getSubscriberId': 71,
                 'Landroid/telephony/TelephonyManager->getDeviceId': 70,
                 'Landroid/telephony/SmsManager->sendTextMessage': 64,
                 'Landroid/telephony/gsm/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/SmsManager->sendMultipartTextMessage': 62,
                 'Landroid/telephony/gsm/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->sendTextMessage': 61,
                 'Landroid/telephony/SmsManager->sendDataMessage': 61,
                 'Landroid/telephony/gsm/SmsManager->getDisplayOriginatingAddress': 60,
                 'Landroid/telephony/gsm/SmsManager->getDisplayMessageBody': 60,
                 'Ldalvik/system/DexClassLoader->loadClass': 50, 'Ldalvik/system/PathClassLoader->loadClass': 50,
                 'Landroid/content/ContentResolver->update': 40, 'Landroid/content/ContentResolver->insert': 30,
                 'Landroid/ContentResolver->query': 20, 'Landroid/content/Intent->setDataAndType': 10
                }

Android Malware Detection based on Useful API Calls and Machine Learning 2018 IEEE First International Conference on Artificial Intelligence and Knowledge Engineering (AIKE)

In [34]:
# you have to cehck whether Aandrogurad call method via '.' or '->'
malicious_apis_2 = {'Ljavax/sql/ConnectionEvent;-><init>':255, 'Ljava/nio/channels/WritableByteChannel;->close':250,
 'Landroid/service/carrier/CarrierService;->stopSelf':245 ,
 'Landroid/app/NativeActivity;->getVolumeControlStream':240, 
 'Landroid/widget/AdapterView;->refreshDrawableState':235, 'Landroid/opengl/Matrix;->getClass':230, 
 'Landroid/widget/MultiAutoCompleteTextView;->saveHierarchyState':225, 
 'Landroid/view/ViewStructure;->setCheckable':220, 'Landroid/text/method/BaseKeyListener;->getInputType':215, 
 'Landroid/provider/MediaStore$Images$Media;->wait':210
}

benign_api_1 = {'Ljava/lang/Object;.<init>':0, 'Ljava/lang/StringBuilder;.toString':5, 'Ljava/lang/StringBuilder;.append':10,
 'Landroid/app/Activity;.<init>':15, 'Ljava/lang/StringBuilder;.<init>':20, 'Ljava/lang/String;.equals':25,
 'Ljava/util/ArrayList;.<init>':30, 'Ljava/util/Iterator;.next':35,
 'Ljava/util/Iterator;.hasNext':40
}

Feature extraction from Android application packages and its usage in machine learning for malware classification. Computer Science Institute of Charles University 2017

In [35]:
malicious_apis_3 = {'Ljava.io.BufferedOutputStream;-><init>':205, 'Ljava.io.FileOutputStream;-><init>':200,
 'Ljava.lang.System;->loadLibrary':195, 
 'Landroid.content.Context;->getPackageManager':190
}

Mining API Calls and Permissions for Android Malware Detection (2014)
The LNM Institute of Information Technology, Jaipur, India 

In [36]:
malicious_apis_4 = {'Landroid/telephony/TelephonyManager;->getCallState':185,
 'Landroid/telephony/TelephonyManager;->getDataActivity':180,
 'Landroid/telephony/TelephonyManager;->getDeviceId':175,
 'Landroid/telephony/TelephonyManager;->getLine1number':170,
 'Landroid/telephony/TelephonyManager;->getNetworkType':165,
 'Landroid/telephony/TelephonyManager;->getSimOperator':160,
 'Landroid/telephony/TelephonyManager;->getSimSerialNumber':155,
 'Landroid/telephony/TelephonyManager;->getSimState':150,
 'Landroid/telephony/TelephonyManager;->getSubscriberId':145
}

## Important Permissions

Android Malware Detection Using Fine-Grained Features 
Hindawi Scientific Programming Volume 2020

In [37]:
#Activity
malicous_activities = {
    'ACCESS_COARSE_LOCATION':250,
    'ACCESS_FINE_LOCATION':240,
    'READ_SMS':230,
    'SEND_SMS':220,
    'READ_PHONE_STATE':210,
    'READ_CONTACTS':200,
    'READ_HISTORY_BOOKMARKS':190,
    'MOUNT_UNMOUNT_FILESYSTEMS':180,
    'RECEIVE_SMS':170
}



In [38]:
#Services
malicious_services = {
    'SEND_SMS':160,
    'READ_SMS':150,
    'ACCESS_COARSE_LOCATION':140,
    'ACCESS_FINE_LOCATION':130,
    'READ_PHONE_STATE':120
}


In [39]:
#BroadCast Reecvier
malicious_recivers = {
    'SEND_SMS':110,
    'READ_SMS':100,
    'ACCESS_COARSE_LOCATION':90,
    'ACCESS_FINE_LOCATION':80,
    'READ_PHONE_STATE':70
}


In [40]:
#Intents
malicious_intents = {
    'BOOT_COMPLETED':60,
    'SMS_RECEIVED':50,
    'BATTERY_CHANGE_ACTION':40,
    'MOUNT_UNMOUNT_FILESYSTEMS':30
}


In [41]:
#Not used but decleared
decleared_permissions={
    'CAMERA':250,
    'RECORD_AUDIO':230,
    'READ_CONTACTS':210,
    'READ_CALL_LOG':200,
    'CALL_PHONE':190,
    'WRITE_CALL_LOG':180,
    'CHANGE_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'READ_CALENDAR':140,
    'ACCESS_WIFI_STATE':130,
    'ACCESS_COARSE_LOCATION':120,
    'ACCESS_FINE_LOCATION':110,
    'READ_PHONE_STATE':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'ACCESS_NETWORK_STATE':60,
    'WRITE_CALENDAR':50,
    'SEND_SMS':40,
    'READ_SMS':30,
    }

In [42]:
#Permissions
malicious_permissions = {
    'CAMERA':250,
    'RECORD_AUDIO':240,
    'READ_CONTACTS':230,
    'READ_CALL_LOG':220,
    'CALL_PHONE':210,
    'WRITE_CALL_LOG':200,
    'CHANGE_WIFI_STATE':190,
    'READ_CALENDAR':180,
    'ACCESS_WIFI_STATE':170,
    'GET_ACCOUNTS':160,
    'WAKE_LOCK':150,
    'GET_TASKS':130,
    'VIBRATE':120,
    'PROCESS_OUTGOING_CALLS':110,
    'SYSTEM_ALERT_WINDOWS':100,
    'WRITE_SETTING':90,
    'WRITE_EXTERNAL_STORAGE':80,
    'READ_EXTERNAL_STORAGE':70,
    'WRITE_CALENDAR':60,
    'ACCESS_NETWORK_STATE':50,
    'READ_HISTORY_BOOKMARKS':40
}